In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import regionmask
import scipy
import matplotlib.pyplot as plt
import numpy as np
import netCDF4

In [2]:
#### Functions necessary ########


########get area of interest based on the shape file #########

def get_aoi(shp, world=True):
    """Takes a geopandas object and converts it to a lat/ lon
    extent 

    Parameters
    -----------
    shp : GeoPandas GeoDataFrame
        A geodataframe containing the spatial boundary of interest
    world : boolean
        True if you want lat / long to represent sinusoidal (0-360 degrees)

    Returns
    -------
    Dictionary of lat and lon spatial bounds
    """

    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    if world:
        aoi_lon[0] = aoi_lon[0] + 360
        aoi_lon[1] = aoi_lon[1] + 360
    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat

############## Get maca file path ##############################

def get_maca_cli(var, model, scenario, year_start,year_end, domain):
    dir_path = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'
    
    run_num = [1] * 20
    run_num[4] = 6  # setting CCSM4 with run 6
    
    time = year_start+'_'+year_end
    
    file_name = ('agg_macav2metdata_' +
             str(variable_name[var]) +
             '_' +
             str(model_name[model]) +
             '_r' +
             str(run_num[model])+'i1p1_' +
             str(scenario_type[scenario]) +
             '_' +
             time + '_' +
             domain + '_daily.nc')
    full_file_path = dir_path + file_name

    return full_file_path

#################### Get the variable ###############################

def get_fut_cli_var(full_path_name):
    ### select the boundary from your watershed #####
    cli_var_xr = xr.open_dataset(full_path_name)
    
    #### get the boundary of your watershed ####
    #### need a get aoi function #######
    
    bounds = get_aoi(wsd)
    
    ##### Select the watershed upto your boundary ##########
    myarr = cli_var_xr.sel(
    #time=slice(start_date, end_date),
    lon=slice(bounds["lon"][0], bounds["lon"][1]),
    lat=slice(bounds["lat"][0], bounds["lat"][1]))
    
    ### create a mask for your study area #####
    
    mynew_mask = regionmask.mask_3D_geopandas(wsd,
                                             myarr.lon,
                                            myarr.lat)
    my_fin_arr = myarr.where(mynew_mask)
    
    ### compute spatial summary in your area #######
    spat_summ = my_fin_arr.mean(["lat","lon"]).mean("region").mean("crs")
    
    
    ###### convert the summary to dataframe ######
    ###############################################
    var_data = spat_summ.to_dataframe().reset_index()
    
    ##############################################
    #var_data.to_csv(var_of_int+".csv")
    return var_data

In [3]:
import os, glob

In [4]:
main_dir = "E:\\WORK\\Future_erosion_prj"
zones = ["high","intermediate","low"]

In [5]:
os.chdir(main_dir)

In [6]:
# os.listdir(main_dir+"\\"+zones[1]+ "\\data")

In [17]:
model_name = ['bcc-csm1-1',
              'bcc-csm1-1-m',
              'BNU-ESM',
              'CanESM2',
              'CCSM4',
              'CNRM-CM5',
              'CSIRO-Mk3-6-0',
              'GFDL-ESM2G',
              'GFDL-ESM2M',
              'HadGEM2-CC365',
              'HadGEM2-ES365',
              'inmcm4',
              'IPSL-CM5A-MR',
              'IPSL-CM5A-LR',
              'IPSL-CM5B-LR',
              'MIROC5',
              'MIROC-ESM',
              'MIROC-ESM-CHEM',
              'MRI-CGCM3',
              'NorESM1-M']

# variable_name = [
# #                 'tasmax',
# #                  'tasmin',
# #                  'pr',
#                  'rsds']

scenario_type = ['historical', 'rcp45', 'rcp85']

# Year start and ends (historical vs projected)
year_start = ['1950', '2006', '2006']
year_end = ['2005', '2099', '2099']




In [8]:
domain = "CONUS"
strt_p = ['1950', '2006', '2006']
end_p = ['2005', '2099', '2099']
zn_shps = ["sfcw.shp","uicw.shp","wlcw.shp"]
n_gen_list = list()
error_indices = {}
for z in range(0,3):
    d1 = main_dir+"\\"+zones[z]
    shp = zn_shps[z]
    wsd = gpd.read_file(d1+"\\data\\"+shp)
    wsd = wsd.to_crs(epsg=4326)
    
    if not os.path.isdir(d1):
        os.makedirs(d1)
    for model in range(0,20):
        d2 = d1+"\\"+model_name[model]
        if not os.path.isdir(d2):
            os.makedirs(d2)
        for scenario in range(0,3):
            d3 = d2+"\\"+scenario_type[scenario]
            if not os.path.isdir(d3):
                os.makedirs(d3)
            d4 = d3 + "\\clidata"
            if not os.path.isdir(d4):
                os.makedirs(d4)
            for var in range(0,1):
                year_start = strt_p[scenario]
                year_end = end_p[scenario]
                try:
                    fullpath = get_maca_cli(var,model,scenario,year_start,year_end,domain)
                except:
                    print("couldnt obtain a full path of MACA for the given combination")
                
                try:
                    vr_data = get_fut_cli_var(fullpath)
                    vr_data.to_csv(d4+"\\"+variable_name[var]+".csv")
                except:
#                     not_generated = {"Model name": model_name[model],
#                                     "Property":variable_name[var],
#                                     "Scenario":scenario_type[scenario],
#                                      "Zone":zones[z]}
                    error_indices[(model_name[model],variable_name[var],
                                  scenario_type[scenario],zones[z])] = True
#                     n_gen_list = n_gen_list.append(not_generated)
                    print("couldnt excecute",zones[z]+"-"+model_name[model]+"-"+scenario_type[scenario]+"-"+variable_name[var]+
                         "\nFullpath generated: "+fullpath)
                    pass
                

In [9]:

# list(range(0,1))
# # for i in 

In [10]:
# print(not_generated)

In [11]:
# n_gen_list.append(not_generated)

In [12]:
#   print("couldnt excecute",zones[z]+"-"+model_name[model]+"-"+scenario_type[scenario]+"-"+var_name[var])

In [13]:
#    error_indices

In [16]:
mods_dir = "C:\\Users\\mugalsamrat.dahal\\OneDrive - Washington State University (email.wsu.edu)\\Paper2\\Weppwatershed"
os.chdir(mods_dir)
from copy_files import copyfiles

In [23]:
import logging
logger = logging.getLogger('ftpuploader')

In [28]:
#### copying files from previous directory 
error_indices = {}
tillage = ["intense","reduced","notill"]
copy_dirs = ["springcreek_present","upper_imbler_present","winn_lake_canyon_present"]

copy_main_dir = "F:\\WORK\\Project_2\\WEPPwatershed\\after_anisotropy\\New_batch_past_and_present_slope_length_200m"


for z in range(0,3):
    for model in range(0,20):
        for scenario in range(0,3):
            for till in range(0,3):
                outd = main_dir+"\\"+zones[z]+"\\"+model_name[model]+"\\"+scenario_type[scenario]+"\\"+tillage[till]
    
                if not os.path.isdir(outd):
                    os.makedirs(outd)
                
                outd_d1 = outd+"\\weppsetup"
                if not os.path.isdir(outd_d1):
                    os.makedirs(outd_d1)
                outd_d2 = outd_d1+"\\wepp"
                
                if not os.path.isdir(outd_d2):
                    os.makedirs(outd_d2)
                
                outd_d3 = outd_d2+"\\output"
                    
                if not os.path.isdir(outd_d3):
                    os.makedirs(outd_d3)
                    
                tmp_copy_dir = copy_main_dir+"\\"+copy_dirs[z]+"_"+tillage[till]+"\\wepp\\runs"
                
                try:
                    copyfiles(tmp_copy_dir, outd_d3, "run", True)
                
                except BaseException as e:
                    
                    error_indices[(model_name[model],scenario_type[scenario],zones[z], tillage[till])] = True
                    logger.error("Failed:" + str(e))
                
                


In [ ]:
#### Now need to make changes to climate file ##########
##change various climate properties in loop
#### Change the number of years in the cli file as well ##########


In [29]:
error_indices

{}

In [ ]:
#### Need to make changes to runfile of each hillslope based on the number of years i would like to run the model ########
#### Make changes to runfile channel file #####

